# Portable long-term zone comfort analytics
Brick metadata schema enables portable data analytics in building science. This IPython notebook uses this technology to evalute long-term thermal comfort over 35 buildings in Mortar Database.  
Aug. 2021

### Initiate Mortar API

In [1]:
# Check pymortar version
!pip show pymortar

Name: pymortar
Version: 2.0.4
Summary: 
Home-page: None
Author: Gabe Fierro
Author-email: gtfierro@cs.berkeley.edu
License: None
Location: /opt/anaconda3/lib/python3.8/site-packages
Requires: requests, brickschema, python-snappy, pyarrow, googleapis-common-protos, rdflib, zstd, pandas
Required-by: 


In [1]:
import pymortar
# connect client to Mortar frontend server
URL  = "https://beta-api.mortardata.org"
client = pymortar.Client(URL)

### Write a query using Brick

In [6]:
# query zone air temperature sensor
query = """SELECT ?sensor ?equip WHERE {{
    ?sensor    rdf:type/rdfs:subClassOf*     brick:Zone_Air_Temperature_Sensor .
    ?sensor    brick:isPointOf ?equip .
}}"""

In [5]:
# get a summary of which sites return results for the query above
qualify_result = client.qualify([query])
qualify_result

         Query_0
artx           8
asmun          0
bixb           0
bwfp          28
chem         140
chemx        134
cont           0
crus          24
dh           128
fdpd          15
gbsf         388
gha_ics      101
giedt         24
gilm           0
hart         126
hick           4
hunt           0
hwc          142
lfh            0
mann          15
math           0
msb            2
msc            0
msd            0
music          3
ols            0
roes          18
scc            0
schm           1
sprl           0
stor         114
swl            0
thur          45
tupp           0
veih          43
vm3a         283
vm3b           0
vmep          18
vmif          84
vmlf           2
vmth          14
well          51
wsrc          31

### Evaluation zones' thermal comfort on one site

In [3]:
from hourly_outlier import *

In [4]:
hourly_outlier?

Signature: hourly_outlier(q, s, a, b, c, d, l, u)
Docstring:
Calculate the percentage of normal occupied time outside a specified temeprature range.
The normal occupied days is Monday to Friday but the occupied time can be specified.

Parameters
----------
q : str
    sparql query using brick metadata schema
s : str
    single qualied site name, using abbreviation
a : str
    start date with format year-month-day, e.g.'2016-1-1'
b : str
    end date with format year-month-day, e.g.'2016-1-31'
c : int
    start hour of normal occupied time with 24-hour clock, e.g. 9
d : int
    end hour of normal occupied time with 24-hour clock, e.g. 17
l : float
    lower bound of the tempearture range, with default F unit
u : float
    upper bound of the temperature range, with default F unit

Returns
----------
p : float
    percentage of the time
File:      ~/Documents/GitHub/mortardata_comfort_evaluation/hourly_outlier.py
Type:      function


In [7]:
ho = hourly_outlier(query, 'artx', '2016-1-1', '2016-12-31', 9, 17, 70, 78)

/Users/ruijisun/Documents/GitHub/mortardata_comfort_evaluation/hourly_outlier.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hr'] = pd.to_datetime(df['time']).dt.hour
/Users/ruijisun/Documents/GitHub/mortardata_comfort_evaluation/hourly_outlier.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wk'] = pd.to_datetime(df['time']).dt.dayofweek
/Users/ruijisun/Documents/GitHub/mortardata_comfort_evaluation/hourly_outlier.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [11]:
ho

,zone,hourly_outlier percentage
3,RM110,0.246783
1,RM107B,0.188815
6,RM100,0.151894
5,RM120,0.142170
2,RM112,0.123632
4,RM115,0.101155
0,RM103,0.098617
7,RM107A,0.071076


### Scipts

In [8]:
# get the brick metadata of the zone air temperature sensor at the first site in qualify_res
meta = client.sparql(query, sites=['artx'])
meta

,sensor,equip,site
0,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....,http://buildsys.org/ontologies/ARTX#VAVRM103,artx
1,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....,http://buildsys.org/ontologies/ARTX#VAVRM107B,artx
2,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....,http://buildsys.org/ontologies/ARTX#VAVRM112,artx
3,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....,http://buildsys.org/ontologies/ARTX#VAVRM110,artx
4,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....,http://buildsys.org/ontologies/ARTX#VAVRM115,artx
5,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....,http://buildsys.org/ontologies/ARTX#VAVRM120,artx
6,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....,http://buildsys.org/ontologies/ARTX#VAVRM100,artx
7,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....,http://buildsys.org/ontologies/ARTX#VAVRM107A,artx


In [9]:
# get dataset with URIs
query_res = client.data_uris(["http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp"])
data = query_res.data

In [10]:
data

,time,value,id
0,2012-04-18 23:45:00+00:00,79.25,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....
1,2012-04-18 23:30:00+00:00,78.75,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....
2,2012-04-18 23:15:00+00:00,79.75,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....
3,2012-04-18 23:00:00+00:00,79.00,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....
4,2012-04-18 22:45:00+00:00,78.50,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....
...,...,...,...
227245,2017-11-30 22:30:00+00:00,71.00,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....
227246,2017-11-30 22:15:00+00:00,71.00,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....
227247,2017-11-30 22:00:00+00:00,71.00,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....
227248,2017-11-30 21:45:00+00:00,71.00,http://buildsys.org/ontologies/ARTX#ARTX.ZONE....


In [93]:
# df.reset_index(drop=True)
# df = data.set_index(['time'])
# df.sort_index(inplace=True, ascending=True)
# df = df[start_date:end_date]

In [95]:
# parse the hour and weekdays info from the datetime column
df['hour'] = pd.to_datetime(df['time']).dt.hour
df['weekday'] = pd.to_datetime(df['time']).dt.dayofweek

<ipython-input-95-f1b87f66776d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = pd.to_datetime(df['time']).dt.hour
<ipython-input-95-f1b87f66776d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weekday'] = pd.to_datetime(df['time']).dt.dayofweek


In [105]:
# create a new dataframe for the specified office hours and weekdays
df_occ = df[(df['hour'] >= a) & (df['hour'] < b) &
            (df['weekday'] >= 0) & (df['weekday'] <= 4)]
df_occ

,time,value,id,hour,weekday
101185,2016-01-20 16:45:00+00:00,72.50,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,16,2
101186,2016-01-20 16:30:00+00:00,72.50,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,16,2
101187,2016-01-20 16:15:00+00:00,72.50,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,16,2
101188,2016-01-20 16:00:00+00:00,72.50,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,16,2
101189,2016-01-20 15:45:00+00:00,72.50,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,15,2
...,...,...,...,...,...
204037,2016-01-21 10:00:00+00:00,70.25,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,10,3
204038,2016-01-21 09:45:00+00:00,70.25,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,9,3
204039,2016-01-21 09:30:00+00:00,69.75,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,9,3
204040,2016-01-21 09:15:00+00:00,70.25,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,9,3


In [108]:
df_out = df_occ[(df_occ['value'] < l) | (df_occ['value'] > u)]
df_out

,time,value,id,hour,weekday
101195,2016-01-20 14:15:00+00:00,71.00,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,14,2
101196,2016-01-20 14:00:00+00:00,70.50,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,14,2
101197,2016-01-20 13:45:00+00:00,70.50,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,13,2
101198,2016-01-20 13:30:00+00:00,71.00,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,13,2
101199,2016-01-20 13:15:00+00:00,71.00,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,13,2
...,...,...,...,...,...
204037,2016-01-21 10:00:00+00:00,70.25,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,10,3
204038,2016-01-21 09:45:00+00:00,70.25,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,9,3
204039,2016-01-21 09:30:00+00:00,69.75,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,9,3
204040,2016-01-21 09:15:00+00:00,70.25,http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp,9,3


In [109]:
p = len(df_out) / len(df_occ)
p

0.8869047619047619

In [125]:
a='Beautiful, is; better*than\nugly'
import re
re.split('; |, |\*|\n',a)[-1]

'ugly'

In [120]:
id

'http://buildsys.org/ontologies/ARTX#ARTX.ZONE.AHU02.RM103.Zone_Air_Temp'

In [127]:
import re
# input sensors' metadata to thermal comfort index calculation package
for id in site_metadata.sensor:
    zone_name = re.split('[.]', id)[-2]
    

RM103
RM107B
RM112
RM110
RM115
RM120
RM100
RM107A
